### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (f'{url}api-key={nyt_api_key}&begin_date={begin_date}'
             + f'&end_date={end_date}&fq={filter_query}&sort={sort}'
             + f'&fl={field_list}')

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    # API results show 10 articles at a time
    page_num = f'&page={page}'
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(f"{query_url}{page_num}").json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f'Checked page {page}')

        # Print the page number that had no results then break from the loop
    except:
        print(f'No results found for page {page}')

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
### COULD NOT USE " Review" because What's love got to do with it starts with 
### Review:.  Used \u2019 with a space after as alternative.
reviews_df['title'] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 ")])

In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df['title'].drop_duplicates()
# titles_list = reviews_df['title']
titles = titles_list.to_list()

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [11]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response_data = requests.get(f"{url}{title}{tmdb_key_string}").json()
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response_data["results"][0]["id"]

        # Make a request for a the full movie details
        query_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        
        # Execute "GET" request with url
        movie_data = requests.get(query_url).json()
        
        # Extract the genre names into a list
        genres = [genre['name'] for genre in movie_data.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [lang['name'] for lang in movie_data.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
        production_countries = [countries['name'] for countries in movie_data.get('production_countries', [])]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({"title": movie_data["title"],
                     "original_title": movie_data["original_title"],
                     "budget": movie_data["budget"],
                     "language": movie_data["original_language"],
                     "homepage": movie_data["homepage"],
                     "overview": movie_data["overview"],
                     "popularity": movie_data["popularity"],
                     "runtime": movie_data["runtime"],
                     "revenue": movie_data["revenue"],
                     "release_date": movie_data["release_date"],
                     "vote_average": movie_data["vote_average"],
                     "vote_count": movie_data["vote_count"],
                     "genre": genres, "spoken_languages": spoken_languages,
                     "production_countries": production_countries})
        
        # Print out the title that was found
        print(f"{title} was found! Yay.")
    except:
        print(f"{title} was not found. Womp Womp.")


The Attachment Diaries was found! Yay.
What’s Love Got to Do With It? was found! Yay.
You Can Live Forever was found! Yay.
A Tourist’s Guide to Love was found! Yay.
Other People’s Children was found! Yay.
One True Loves was found! Yay.
The Lost Weekend: A Love Story was found! Yay.
A Thousand and One was found! Yay.
Your Place or Mine was found! Yay.
Love in the Time of Fentanyl was found! Yay.
Pamela, a Love Story was found! Yay.
In From the Side was found! Yay.
After Love was found! Yay.
Alcarràs was found! Yay.
Nelly & Nadine was found! Yay.
Lady Chatterley’s Lover was found! Yay.
The Sound of Christmas was found! Yay.
The Inspection was found! Yay.
Bones and All was found! Yay.
My Policeman was found! Yay.
About Fate was found! Yay.
Waiting for Bojangles was found! Yay.
I Love My Dad was found! Yay.
A Love Song was found! Yay.
Alone Together was found! Yay.
Art of Love was found! Yay.
The Wheel was found! Yay.
Thor: Love and Thunder was found! Yay.
Both Sides of the Blade was found

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list, indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.791,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Espa\u00f1ol"
        ],
        "production_countries": [
            "Argentina"
        ]
    },

In [13]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
movie_review_df = pd.merge(reviews_df, tmdb_df, on="title")

In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genre','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ['[',']',"'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    movie_review_df = movie_review_df.astype({column:str}, errors='raise')

    # Loop through characters to remove
    for character in characters_to_remove:
        movie_review_df[column] = movie_review_df[column].str.replace(character,"")

# Display the fixed DataFrame
movie_review_df.head(1)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.791,102,0,2021-10-07,4.0,2,"Drama, Mystery, Thriller, Horror",Español,Argentina


In [16]:
# Drop "byline.person" column
movie_review_df = movie_review_df.drop("byline.person", axis=1)

In [17]:
# Delete duplicate rows and reset index
movie_review_df = movie_review_df.drop_duplicates()
movie_review_df = movie_review_df.reset_index(drop=True)

In [18]:
# Export data to CSV without the index
movie_review_df.to_csv("output/collected_data.csv", index=False, header=True)